<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Retrieval Augmented Generation (RAG) Baseline
## Overview  
In this time, we will check the baseline code.
The goal of this project is to provide students with hands-on experience in handling and enhancing Large Language Models (LLMs) provided by [**Upstage**](https://www.upstage.ai) (Solar).

You can use any engineering method for improving benchmark performance excluding direct training (Fine-tuning).

*Collecting data directly related to the test set is considered cheating e.g., using MMLU-pro dataset or EWHA.pdf for KB*

# Baseline

In [ ]:
#!pip3 install -qU python-dotenv PyPDF2 langchain langchain-community langchain-core langchain-text-splitters langchain_upstage oracledb python-dotenv

In [ ]:
# Additional Contents : https://wikidocs.net/253106

In [51]:
# set parameters

file = open("info/api.txt", "r")
api_key = file.read()
file.close()

file = open("info/datapath.txt", "r")
data_path = file.read()
file.close()

file = open("info/resultspath.txt", "r")
results_path = file.read()
file.close()

In [3]:
from langchain_upstage import UpstageEmbeddings

# 쿼리 전용 임베딩 모델
query_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-query")

# 문장 전용 임베딩 모델
passage_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-passage")

In [4]:
# funcion to extract an answer from response

import re

def extract_answer(response):
    """
    extracts the answer from the response using a regular expression.
    expected format: "[ANSWER]: (A) convolutional networks"

    if there are any answers formatted like the format, it returns None.
    """
    pattern = r"\[ANSWER\]:\s*\((A|B|C|D|E)\)"  # Regular expression to capture the answer letter and text
    match = re.search(pattern, response)

    if match:
        return match.group(1) # Extract the letter inside parentheses (e.g., A)
    else:
        return extract_again(response)

def extract_again(response):
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"
    match = re.search(pattern, response)
    if match:
        return match.group(0)
    else:
        return None

## 1. build DB

In [5]:
from langchain_upstage import UpstageLayoutAnalysisLoader
import os

UPSTAGE_API_KEY = api_key

layzer = UpstageLayoutAnalysisLoader(api_key=UPSTAGE_API_KEY,file_path=os.path.join(data_path, 'ewha.pdf'), output_type="text")

docs = layzer.load()  # or layzer.lazy_load()

/home/aix7101/anaconda3/envs/yoonnlp/lib/python3.10/site-packages/langchain_upstage/layout_analysis.py:135: UserWarning: UpstageLayoutAnalysisLoader is deprecated.Please use langchain_upstage.document_parse.UpstageDocumentParseLoader instead.
  warnings.warn(
/home/aix7101/anaconda3/envs/yoonnlp/lib/python3.10/site-packages/langchain_upstage/layout_analysis_parsers.py:160: UserWarning: UpstageLayoutAnalysisParser is deprecated.Please use langchain_upstage.document_parse_parsers.UpstageDocumentParseParser instead.
  warnings.warn(


## 2. Text Split

In [36]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

chunk_size, chunk_overlap = 500, 100

# 2. Split
text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap, language=Language.HTML
)
splits = text_splitter.split_documents(docs)
print("Splits:", len(splits))

Splits: 119


In [37]:
print(splits[0])
len(splits[0].page_content)

page_content='이화여자대학교 학칙1946. 8. 15. 제정
2017. 8. 16. 개정제1장 총칙제1조(목적) 본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과
그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을 도야하여 국가와 인류사회의 발전에
공헌할 수 있는 지도여성을 양성함을 목적으로 한다.제2조(명칭) 본교는 이화여자대학교라 부른다.
제3조(위치) 본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)제2장 편제제4조(대학 및 대학원) ① 본교에는 다음 각 호의 대학을 둔다.1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범
대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각
대학”이라 한다) (개정 2016.6.16.)
2. 호크마(HOKMA)교양대학
② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대
학원, 디자인대학원' metadata={'total_pages': 54}


500

## 3. query-context matching

In [38]:
# read samples.csv file
import pandas as pd

def read_data(data_path):
    data = pd.read_csv(data_path)
    prompts = data['prompts']
    answers = data['answers']
    # returns two lists: prompts and answers
    return prompts, answers

In [39]:
prompts, answers = read_data(os.path.join(data_path, 'test_own_ewha.csv'))

In [40]:
testdata = pd.read_csv(data_path+'test_own_ewha.csv')

In [41]:
nowtest = pd.DataFrame(columns=['index', 'embed_ques', 'question', 'prompts', 'answers', 'top1', 'top2', 'top3', 'predict', 'ground' ])

for index, row in testdata.iterrows():
    #if index % 100 != 0 : continue # 일단 실험할 땐 100개 단위로 끊어서 가져옴
    q = row.prompts
    a = row.answers
    question = q.partition('(A)')[0]
    question = question.partition(') ')[2]
    q = q.partition(') ')[2]
    try : 
        embedded_query = query_embeddings.embed_query(question) # 질문만 받아와서 embedding 하기
        nowtest.loc[len(nowtest)] = {'index':index, 'embed_ques' : embedded_query, 'question' : question, 'prompts' : q, 'answers' : a, 'top1': None, 'top2': None, 'top3': None, 'predict': None, 'gound': None }

    except :
        print(f'pass: {index}')
        continue


In [42]:
nowtest

,index,embed_ques,question,prompts,answers,top1,top2,top3,predict,ground
0,0,"[-0.005329132080078125, 0.0017337799072265625,...",영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,(D),None,None,None,None,NaN
1,1,"[-0.00460052490234375, -0.0174713134765625, -0...",각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n,각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n(A) 이학사\n(B...,(E),None,None,None,None,NaN
2,2,"[0.007404327392578125, -0.0216522216796875, 0....",부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n,부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n(A) 현...,(B),None,None,None,None,NaN
3,3,"[-0.002162933349609375, -0.039215087890625, -0...",학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n,학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n(A) 시험 직후...,(B),None,None,None,None,NaN
4,4,"[0.0089111328125, -0.0259552001953125, -0.0064...",학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,(C),None,None,None,None,NaN
5,5,"[0.0016117095947265625, -0.019683837890625, -0...",학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n,학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n(A) 서...,(D),None,None,None,None,NaN
6,6,"[-0.018157958984375, -0.0157318115234375, -0.0...",학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?\n,학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?...,(C),None,None,None,None,NaN
7,7,"[0.0029163360595703125, -0.018707275390625, -0...",학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,(D),None,None,None,None,NaN
8,8,"[-5.4895877838134766e-05, -0.01477813720703125...",학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,(C),None,None,None,None,NaN
9,9,"[-0.002346038818359375, 0.00176239013671875, -...",학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건은 무엇입니까?\n,학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건은 무엇입니까?\n(A...,(B),None,None,None,None,NaN


## 4. Prompt engineering

In [43]:
############# first PREDICTION ##########

from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    [Question]에 대한 정답을 출력해. 주어진 [Context]에서 관련된 정보를 찾아.
    
    정확하게 정답만 답변해.
    그리고 답변이 정확한지 한 번 더 확인해
    ---
    
    [Question] {question}
    [Context] {context}
    [Answer]
        
    '''

)
ko_chain = prompt_template | llm


In [45]:
import numpy as np

chunk = []
for index in range(0,len(splits)) : # context 받아오기
    chunk.append(splits[index].page_content)

embedded_documents = passage_embeddings.embed_documents(chunk)

responses = []
for idx, row in nowtest.iterrows() : # 질문 받아오기 

    embed_ques= row.embed_ques

    # 유사도 기준 내림차순 정렬
    sorted_idx = (np.array(embed_ques) @ np.array(embedded_documents).T).argsort()[::-1]

    nowtest.loc[idx, 'top1'] = chunk[sorted_idx[0]]
    nowtest.loc[idx, 'top2'] = chunk[sorted_idx[1]]
    nowtest.loc[idx, 'top3'] = chunk[sorted_idx[2]]

    response = ko_chain.invoke({"question": row.prompts, "context": chunk[sorted_idx[0]]}) # top1에 대해서는 바로 답변 출력
    responses.append(response.content)

    nowtest.loc[idx, 'predict'] = response.content

responses



['(D) 3학점',
 '(E) 인공지능학사',
 '(B) 구 학칙에 의하여 재학생이 졸업하는 연도까지 존속한다',
 '학생 포털에서 본인의 성적을 조회할 수 있는 시기는 성적 입력 마감 후입니다.\n[Answer]: (B) 성적 입력 마감 후',
 '(D) 학술의 응용방법 연구',
 '정답은 (D) 강남구입니다.',
 '(C) 학생총회 요청',
 '(A) 건축학과는 추가 설계 실습을 포함하므로 더 긴 연한이 필요하다.',
 '(E) 중대한 질병으로 인한 휴학: 총장의 판단에 따라 연장 가능',
 '(B) 전공별 종합시험 합격',
 '(B) 과제 제출 기한 초과',
 '(C) 독립된 학과로 운영된다.',
 '(E) 휴학 기간이 만료된 경우',
 '(E) 종강일',
 '(A) 신입생 첫 학기',
 '(C) 15시간 이상',
 '(E) 특정 종교 교리 확립',
 '(B) 통합된 모집단위는 대학장이 결정한다.',
 '(C) 약학대학',
 'B',
 '(C) 학기 중간에 전과를 신청한 경우',
 '(D) 주말',
 '(B) 제1전공의 졸업요건 학점을 충족해야 한다.',
 '(C) 24학점',
 '(A) 수업 중 휴대폰 사용']

In [46]:
nowtest

,index,embed_ques,question,prompts,answers,top1,top2,top3,predict,ground
0,0,"[-0.005329132080078125, 0.0017337799072265625,...",영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,(D),2.29.)② 제1항의 규정에 불구하고 총장은 교과과정상의 필요에 따라 특정 학과 ...,"이수의 단위는 학점으로 하되, 1학기간 15시간 이상의 수업을 1학점으\n로 한다....",교에서 학점을 취득한 경우\n2. 입학 전 국내외의 고등학교와 「고등교육법」 제2조...,(D) 3학점,NaN
1,1,"[-0.00460052490234375, -0.0174713134765625, -0...",각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n,각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n(A) 이학사\n(B...,(E),라 한다) (개정 2016.6.16.)\n2. 호크마(HOKMA)교양대학\n② 본교...,7\n 건강과학대학 간호학부 체육과학부 간호학 전공 78\n 글로벌건강간호학 전공 ...,전공결정 신청 절차는 총장이 따로 정한다. (신설\n2015.9.18.)[제목개정 ...,(E) 인공지능학사,NaN
2,2,"[0.007404327392578125, -0.0216522216796875, 0....",부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n,부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n(A) 현...,(B),"술대학 중 회화과의\n“동양화전공, 서양화전공, 조소전공”과 약학대학 약학과의 “약...","60, 1971. 12. 31 개정)① 본 학칙은 1972년 3월 1일부터 시행한다...",시점에서 이를 폐지한다.부칙(2006. 5. 11 개정)\n이 학칙은 2006년 5...,(B) 구 학칙에 의하여 재학생이 졸업하는 연도까지 존속한다,NaN
3,3,"[-0.002162933349609375, -0.039215087890625, -0...",학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n,학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n(A) 시험 직후...,(B),41조(성적불량학생에 대한 조치) ① 학기말의 평균성적이 1.60 미만인 학생은 지...,94.9.6)\n③ 낙제한 교과목이 필수과목인 때에는 재수강하여야 하고 선택과목인 ...,학기에 등록\n하여 3학점까지 취득할 수 있다. (개정 2003.10.10)\n⑥ ...,학생 포털에서 본인의 성적을 조회할 수 있는 시기는 성적 입력 마감 후입니다.\n[...,NaN
4,4,"[0.0089111328125, -0.0259552001953125, -0.0064...",학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,(C),이화여자대학교 학칙1946. 8. 15. 제정\n2017. 8. 16. 개정제1장 ...,입학할 수 있는 자는 다음 각 호의 1에 해당하는 여자로 한다. (개정 2003.2...,행일) 이 학칙은 2003년 3월 1일부터 시행한다.\n② (제적학생 등의 구제를 ...,(D) 학술의 응용방법 연구,NaN
5,5,"[0.0016117095947265625, -0.019683837890625, -0...",학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n,학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n(A) 서...,(D),6조(대학원 학칙 등) 각 대학원의 학칙은 따로 정한다. (개정 2000.2.1)\...,"교육학과, 유아\n교육과, 초등교육과 및 수학교육과의 재적생으로 본다.부칙(학무25...",이화여자대학교 학칙1946. 8. 15. 제정\n2017. 8. 16. 개정제1장 ...,정답은 (D) 강남구입니다.,NaN
6,6,"[-0.018157958984375, -0.0157318115234375, -0.0...",학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?\n,학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?...,(C),① 정기휴업일은 다음과 같다.1. 하기방학 및 동기방학\n2. 개교기념일(5월 31...,학기) ① 학년은 3월 1일부터 다음해 2월 말일까지로 하고 이를 제1학기와 제2학...,때에는 「고\n등교육법 시행령」 제29조제3항의 규정이 정하는 범위내에서 허가할 수...,(C) 학생총회 요청,NaN
7,7,"[0.0029163360595703125, -0.018707275390625, -0...",학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,(D),"으로 하며, 약학대\n학은 6년으로 하되 다른 학과 또는 학부 등에서 이수하는 기초...",에는 즉시 총장에게 보고하여야 한다.\n(개정 1996.2.15)제21조(등록) 학...,때에는 「고\n등교육법 시행령」 제29조제3항의 규정이 정하는 범위내에서 허가할 수...,(A) 건축학과는 추가 설계 실습을 포함하므로 더 긴 연한이 필요하다.,NaN
8,8,"[-5.4895877838134766e-05, -0.01477813720703125...",학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,(C),"총장이 지정하는\n학부, 학과 또는 전공에 있어서는 이를 1년으로 한다. (개정 1...",때에는 「고\n등교육법 시행령」 제29조제3항의 규정이 정하는 범위내에서 허가할 수...,. (신설 2015.9.18.)이화여자대학교 학칙⑧ 제4항에서 정한 휴학기간이 만료...,(E) 중대한 질병으로 인한 휴학: 총장의 판단에 따라 연장 가능,NaN
9,9,"[-0.002346038818359375, 0.00176239013671875, -...",학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건은 무엇입니까?\n,학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건은 무엇입니까?\n(A...,(B),석사학위과정 수료에 필요한 학점에 포함시킬 수 있다.\n제50조(졸업) ① 이 학칙...,2.29.)② 제1항의 규정에 불구하고 총장은 교과과정상의 필요에 따라 특정 학과 ...,④ 휴학 중에는 계절학기 이수 등으로 학점을 취득하여 졸업요건을 충족하더라도 졸업할...,(B) 전공별 종합시험 합격,NaN


In [61]:
######### 정답 1차 확인 + wrong 뽑아내기 ######

# print accuracy

cnt = 0
wrong = []
for idx, (answer, response) in enumerate(zip(answers, nowtest['predict'])):
    print("-"*10)
    generated_answer = extract_answer(response)
    print(response)
    # check
    if generated_answer:
        print(f"idx: {idx} | generated answer: {generated_answer}, answer: {answer}")
    else:
        print("extraction fail")

    if generated_answer == None:
        wrong.append(idx)
        continue
    
    if generated_answer in answer:
        cnt += 1
    else : 
        wrong.append(idx)
        

print()
print(f"acc: {(cnt/len(answers))*100}%")
print()
print(wrong)

----------
(D) 3학점
idx: 0 | generated answer: D, answer: (D)
----------
(E) 인공지능학사
idx: 1 | generated answer: E, answer: (E)
----------
(B) 구 학칙에 의하여 재학생이 졸업하는 연도까지 존속한다
idx: 2 | generated answer: B, answer: (B)
----------
학생 포털에서 본인의 성적을 조회할 수 있는 시기는 성적 입력 마감 후입니다.
[Answer]: (B) 성적 입력 마감 후
idx: 3 | generated answer: B, answer: (B)
----------
[설명과 Answer]

관련 정보는 학칙의 제1조(목적)에서 찾을 수 있습니다. 학칙의 목적은 "이화여자대학교는 기독교정신에 입각하여 교육이념과 교육목적을 달성함으로써 국가와 인류사회의 발전에 공헌하는 지도적인 여성인재를 양성함을 목적으로 한다."라고 명시되어 있습니다.

(A) 기독교정신을 바탕으로 교육
(B) 지도여성 양성
(C) 지역사회의 봉사
(D) 학술의 응용방법 연구
(E) 국제사회 협력
(F) 국가와 인류사회의 발전에 공헌

정답: (D) 학술의 응용방법 연구
idx: 4 | generated answer: A, answer: (C)
----------
정답은 (D) 강남구입니다.
idx: 5 | generated answer: D, answer:  (D)
----------
(C) 학생총회 요청
idx: 6 | generated answer: C, answer:  (C)
----------
[설명과 Answer]

해당 정보를 찾을 수 있는 부분은 [Context]의 제23조(수업연한)입니다. 이 조항에 따르면, 건축학과는 다른 학과에 비해 1년 더 긴 5년의 수업연한이 필요합니다. 약학대학도 다른 학과에 비해 2년 더 긴 6년의 수업연한이 필요하며, 이 중 2년은 기초 및 소양 교육, 나머지 4년은 전공 교육으로 구성됩니다.

정답은 (

In [64]:
############# Second PREDICTION ##########

from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    [Question]에 대한 정답을 출력해. 주어진 [Context]에서 관련된 정보를 찾아.
    
    어느 부분에서 정보를 찾았는지 설명해.
    정확한 답변을 도출해.
    ---
    
    [Question] {question}
    [Context] {context} {context}
    [Answer]
    [설명]
        
    '''

)
ko_chain = prompt_template | llm


response2 = []
for idx, row in nowtest.iterrows() : ############### 일단 지금은 100개 단위로 띄엄띄엄 test 중
    if idx not in wrong : continue
        
    response = ko_chain.invoke({"question": row.prompts, "context": row.top1, "context": row.top2}) # 선지 전까지 받아오기
    response2.append(response.content)
    nowtest.loc[idx, 'predict'] = response.content

response2

['[Answer] (D) 학술의 응용방법 연구\n\n[설명] 질문에 대한 답을 찾기 위해, 이화여자대학교 학칙 제1조를 참고했습니다. 학칙 제1조는 다음과 같이 명시하고 있습니다: "이 학교는 기독교 정신을 바탕으로 교육하여 지도자적 여성 인재를 양성하며, 지역 사회와 인류 사회에 기여하는 것을 목적으로 한다."\n\n이 내용을 바탕으로, 각 선택지를 평가해보겠습니다:\n\n(A) 기독교 정신을 바탕으로 교육 - 학칙에 명시된 내용과 일치합니다.\n(B) 지도여성 양성 - 학칙에 명시된 내용과 일치합니다.\n(C) 지역사회의 봉사 - 학칙에 명시된 내용과 일치합니다.\n(D) 학술의 응용방법 연구 - 학칙에 명시된 내용과 직접적으로 일치하지 않습니다.\n(E) 국제사회 협력 - 학칙에 명시된 내용과 일치합니다.\n(F) 국가와 인류사회의 발전에 공헌 - 학칙에 명시된 내용과 일치합니다.\n\n따라서, 학칙에 명시된 내용과 직접적으로 일치하지 않는 선택지는 (D) 학술의 응용방법 연구입니다.',
 '[Answer]: B\n\n[Explanation]: 주어진 문맥에서, 제23조(수업연한)에서 건축학과와 약학대학의 수업연한이 다른 이유를 설명하고 있습니다. 건축학과는 5년, 약학대학은 6년으로 되어 있습니다. 옵션 B는 약학대학이 국제 인증 기준에 따라 6년이 요구된다고 설명하고 있으며, 이는 문맥에서 언급된 내용과 일치합니다. 따라서, 옵션 B가 올바른 설명입니다.',
 '정답: (A) 임신 및 육아로 인한 휴학: 2년\n\n설명:\n해당 문제는 학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌 것을 찾는 문제입니다. 학칙에 따르면 휴학기간은 통산하여 3년(건축학전공의 경우 4년, 의예과의 경우 3학기)을 초과할 수 없다고 명시되어 있습니다.\n\n각 선택지에 대한 기간은 다음과 같습니다:\n(A) 임신 및 육아로 인한 휴학: 2년\n(B) 창업으로 인한 휴학: 2년\n(C) 군복무로 인한 휴학: 의무 복무 기간\n(D) 해외

In [63]:
######### 정답 2차 확인 + wrong 뽑아내기 ######

# print accuracy

cnt = 0
wrong2 = []
for idx, (answer, response) in enumerate(zip(answers, nowtest['predict'])):
    print("-"*10)
    generated_answer = extract_answer(response)
    print(response)
    # check
    if generated_answer:
        print(f"idx: {idx} | generated answer: {generated_answer}, answer: {answer}")
    else:
        print("extraction fail")

    if generated_answer == None:
        wrong2.append(idx)
        continue
    
    if generated_answer in answer:
        cnt += 1
    else : 
        wrong2.append(idx)
        

print()
print(f"acc: {(cnt/len(answers))*100}%")
print()
print(wrong2)

----------
(D) 3학점
idx: 0 | generated answer: D, answer: (D)
----------
(E) 인공지능학사
idx: 1 | generated answer: E, answer: (E)
----------
(B) 구 학칙에 의하여 재학생이 졸업하는 연도까지 존속한다
idx: 2 | generated answer: B, answer: (B)
----------
학생 포털에서 본인의 성적을 조회할 수 있는 시기는 성적 입력 마감 후입니다.
[Answer]: (B) 성적 입력 마감 후
idx: 3 | generated answer: B, answer: (B)
----------
[설명과 Answer]

관련 정보는 학칙의 제1조(목적)에서 찾을 수 있습니다. 학칙의 목적은 "이화여자대학교는 기독교정신에 입각하여 교육이념과 교육목적을 달성함으로써 국가와 인류사회의 발전에 공헌하는 지도적인 여성인재를 양성함을 목적으로 한다."라고 명시되어 있습니다.

(A) 기독교정신을 바탕으로 교육
(B) 지도여성 양성
(C) 지역사회의 봉사
(D) 학술의 응용방법 연구
(E) 국제사회 협력
(F) 국가와 인류사회의 발전에 공헌

정답: (D) 학술의 응용방법 연구
idx: 4 | generated answer: A, answer: (C)
----------
정답은 (D) 강남구입니다.
idx: 5 | generated answer: D, answer:  (D)
----------
(C) 학생총회 요청
idx: 6 | generated answer: C, answer:  (C)
----------
[설명과 Answer]

해당 정보를 찾을 수 있는 부분은 [Context]의 제23조(수업연한)입니다. 이 조항에 따르면, 건축학과는 다른 학과에 비해 1년 더 긴 5년의 수업연한이 필요합니다. 약학대학도 다른 학과에 비해 2년 더 긴 6년의 수업연한이 필요하며, 이 중 2년은 기초 및 소양 교육, 나머지 4년은 전공 교육으로 구성됩니다.

정답은 (

## groundedness check

### 질문할 때마다 답이 달라져서 안 할 거임 

In [ ]:
import os
from langchain_upstage import UpstageGroundednessCheck # langchain_upstage==0.1.3
 
groundedness_check = UpstageGroundednessCheck()
 
request_input = {
    "context": f"Mauna Kea is an inactive volcano on the island of Hawai'i. Its peak is 4,207.3 m above sea level, making it the highest point in Hawaii and second-highest peak of an island on Earth.",
    "answer": "Mauna Kea is 5,207.3 meters tall.",
}
response = groundedness_check.invoke(request_input)
print(response)

## Check Accuracy

In [96]:
# print accuracy

cnt = 0
wrong = []
for idx, (answer, response) in enumerate(zip(answers, responses)):
    print("-"*10)
    generated_answer = extract_answer(response)
    print(response)
    # check
    if generated_answer:
        print(f"idx: {idx} | generated answer: {generated_answer}, answer: {answer}")
    else:
        print("extraction fail")

    if generated_answer in answer:
        cnt += 1
    else : 
        wrong.append(idx)
        if generated_answer == None:
            continue

print()
print(f"acc: {(cnt/len(answers))*100}%")

----------
(C) 2학점
idx: 0 | generated answer: C, answer: (D)
----------
H
idx: 1 | generated answer: H, answer: (E)
----------
(B) 구 학칙에 의하여 재학생이 졸업하는 연도까지 존속한다
idx: 2 | generated answer: B, answer: (B)
----------
B. 성적 입력 마감 후
idx: 3 | generated answer: B, answer: (B)
----------
(D) 학술의 응용방법 연구
idx: 4 | generated answer: D, answer: (C)
----------
정답은 (D) 강남구입니다.
idx: 5 | generated answer: D, answer:  (D)
----------
D) 대규모 학내 행사
idx: 6 | generated answer: D, answer:  (C)
----------
(A) 건축학과는 추가 설계 실습을 포함하므로 더 긴 연한이 필요하다.
idx: 7 | generated answer: A, answer:  (D)
----------
D) 해외 봉사로 인한 휴학: 3년
idx: 8 | generated answer: D, answer:  (C)
----------
(B) 전공별 종합시험 합격
idx: 9 | generated answer: B, answer:  (B)
----------
(D) 교과목 담당교수의 허가 없이 수강 취소
idx: 10 | generated answer: D, answer:  (B)
----------
(C) 독립된 학과로 운영된다.
idx: 11 | generated answer: C, answer:  (C)
----------
(D) 타교에 입학한 경우
idx: 12 | generated answer: D, answer:  (B)
----------
(E) 종강일입니다.
idx: 13 | generated answer: E, answer: 

In [97]:
wrong

[0, 1, 4, 6, 7, 8, 10, 12, 15, 19, 20]